In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backpor

In [2]:
from pyspark import SparkFiles
url = ""
spark.sparkContext.addFile('https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Major_Appliances_v1_00.tsv.gz')
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   16199106|R203HPW78Z7N4K|B0067WNSZY|     633038551|FGGF3032MW Galler...|Major Appliances|          5|            0|          0|   N|                Y|If you need a new...|What a great stov...| 2015-08-31|
|         US|   16374060|R2EAIGVLEALSP3|B002QSXK60|     811766671|Best Hand Clothes...|Major Appliances|          5|    

In [3]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R203HPW78Z7N4K|          5|            0|          0|   N|                Y|
|R2EAIGVLEALSP3|          5|            1|          1|   N|                Y|
|R1K1CD73HHLILA|          5|            0|          0|   N|                Y|
|R2KZBMOFRMYOPO|          5|            1|          1|   N|                Y|
| R6BIZOZY6UD01|          5|            0|          0|   N|                Y|
|R1MCXZFNF8E7Y0|          1|            0|          0|   N|                Y|
|R3EMB3E3ODR6BW|          5|            2|          2|   N|                Y|
| RJTONVTTOPJ5S|          5|            0|          0|   N|                Y|
|R21U5QZ2CQECUM|          4|            0|          0|   N|                Y|
| RL2BBC51H89DH|          4|            0|          0|   N|     

In [4]:
# Convert star_rating column to integer
from pyspark.sql.types import IntegerType
vine_df = vine_df.withColumn("star_rating",vine_df.star_rating.cast(IntegerType()))
vine_df.dtypes

[('review_id', 'string'),
 ('star_rating', 'int'),
 ('helpful_votes', 'int'),
 ('total_votes', 'int'),
 ('vine', 'string'),
 ('verified_purchase', 'string')]

In [15]:
# Filter the data and create a new DataFrame or table to retrieve all the rows where the total_votes count is equal to or greater than 20

helpful_votes_df = vine_df.filter("total_votes>=20")
helpful_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1QXNQBTH7LIUB|          3|            9|         22|   N|                Y|
|R3BC75VKJK6LNV|          4|           83|         89|   N|                Y|
| R5XKK92G5N9FU|          1|           18|         22|   N|                N|
| RYMOWYW38WKOB|          5|           35|         35|   N|                N|
| RYHZAVESD0T37|          1|           39|         45|   N|                Y|
|R19Y5VBOF3BQOG|          1|           26|         35|   N|                Y|
| R8V9F3139Z1WZ|          1|           30|         34|   N|                N|
|R3RNY5OLY451GS|          1|           25|         30|   N|                Y|
|R16LPVYSJOAH1T|          1|           20|         20|   N|                N|
|R2NLL7ISMM5QYQ|          4|           89|         94|   N|     

In [16]:
#Filter the new DataFrame created in Step 1 and create a new DataFrame to retrieve all the rows where the number of helpful_votes divided by total_votes is equalor greater than 50%
new_helpful_votes_df = helpful_votes_df.filter("helpful_votes/total_votes>=.5")
new_helpful_votes_df.show(truncate=False)

+--------------+-----------+-------------+-----------+----+-----------------+
|review_id     |star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3BC75VKJK6LNV|4          |83           |89         |N   |Y                |
|R5XKK92G5N9FU |1          |18           |22         |N   |N                |
|RYMOWYW38WKOB |5          |35           |35         |N   |N                |
|RYHZAVESD0T37 |1          |39           |45         |N   |Y                |
|R19Y5VBOF3BQOG|1          |26           |35         |N   |Y                |
|R8V9F3139Z1WZ |1          |30           |34         |N   |N                |
|R3RNY5OLY451GS|1          |25           |30         |N   |Y                |
|R16LPVYSJOAH1T|1          |20           |20         |N   |N                |
|R2NLL7ISMM5QYQ|4          |89           |94         |N   |Y                |
|R3FAFI6Q0YL37W|1          |30           |35         |N   |Y    

In [41]:
# Filter the DataFrame created in Step 2 and create new DataFrame that retreives all rows where a review was written as part of the Vine program
vine_helpful_votes_df = new_helpful_votes_df.filter("vine=='Y'")
vine_helpful_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R20O97P5V9CZ21|          3|           21|         27|   Y|                N|
| RCNB7YGLWKCV5|          5|           23|         30|   Y|                N|
| RX1ZUVGBH3HKC|          5|          814|        848|   Y|                N|
| RRS5AGL10DYU3|          3|           25|         29|   Y|                N|
| R5YFC5G8V2B3I|          5|          104|        117|   Y|                N|
|R1IDQ0G1S7J9TN|          5|          178|        196|   Y|                N|
| RKXNZX22HQZEE|          3|          182|        192|   Y|                N|
|R2K45IY0NO70ZI|          5|          116|        122|   Y|                N|
| RQXRI4UKDX877|          5|           20|         24|   Y|                N|
|R1L2R8OHO171YG|          5|           66|         68|   Y|     

In [40]:
# Filter the DataFrame created in Step 2 and create new DataFrame that retreives all rows where a review was written as part of the Vine program
nonvine_helpful_votes_df = new_helpful_votes_df.filter("vine=='N'")
nonvine_helpful_votes_df.count()

4957

In [33]:
# Determine the total number of reviews, the number of 5-star reviews, and the percentage of 5-star reviews for the two types of review (paid vs unpaid).
print('Total number of reviews:')
df.count()

Total number of reviews:


96901

In [36]:
print('Total number of five-star reviews:')
five_star = df.filter("star_rating=5")
five_star.count()

Total number of five-star reviews:


49704

In [66]:
print('percentage of 5-star reviews for vine reviews')
value_of_5star_vine=vine_helpful_votes_df.filter("star_rating=5")
value_of_5star_vine=value_of_5star_vine.count()
value_of_vine=vine_helpful_votes_df.count()
percentage=(value_of_5star_vine/value_of_vine)
print((percentage*100))

percentage of 5-star reviews for both vine and non-vine reviews
51.42857142857142


In [68]:
print('percentage of 5-star reviews for non-vine reviews')
value_of_5star_nonvine=nonvine_helpful_votes_df.filter("star_rating=5")
value_of_5star_nonvine=value_of_5star_nonvine.count()
value_of_nonvine=nonvine_helpful_votes_df.count()
percentage=(value_of_5star_nonvine/value_of_nonvine)
print((percentage*100))

percentage of 5-star reviews for non-vine reviews
39.60056485777688
